# Data exploration

[MovieLens 100K dataset](https://grouplens.org/datasets/movielens/100k/)


In [284]:
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

In [174]:
MANUAL_SEED: int = 42

## Data loading


In [56]:
def load_genres(path: str) -> list[str]:
    return pd.read_csv(
        os.path.join(path, "u.genre"),
        sep="|",
        header=None,
        names=["name", "genre_idx"],
        encoding="ISO-8859-1",
    )["name"].tolist()


def load_items(path: str, genres: list[str]) -> pd.DataFrame:
    return pd.read_csv(
        os.path.join(path, "u.item"),
        sep="|",
        header=None,
        names=[
            "movie_id",
            "movie_title",
            "release_date",
            "video_release_date",
            "IMDb_URL",
            *genres,
        ],
        encoding="ISO-8859-1",
    )


def load_users(path: str) -> pd.DataFrame:
    return pd.read_csv(
        os.path.join(path, "u.user"),
        sep="|",
        header=None,
        names=["user_id", "age", "gender", "occupation", "zip_code"],
        encoding="ISO-8859-1",
    )


def load_connections(path: str) -> pd.DataFrame:
    return pd.read_csv(
        os.path.join(path, "u.data"),
        sep="\t",
        names=["user_id", "item_id", "rating", "timestamp"],
        engine="python",
    )


genres = load_genres("../data/interim/ml-100k/")
raw_connections_df = load_connections("../data/interim/ml-100k/")
raw_items_df = load_items("../data/interim/ml-100k/", genres)
raw_users_df = load_users("../data/interim/ml-100k/")


num_users = len(raw_users_df)
num_items = len(raw_items_df)
print(f"{num_users=}\n{num_items=}")
raw_connections_df.head()

num_users=943
num_items=1682


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


## Data preprocessing

In [13]:
users_df = raw_users_df.copy()

In [47]:
def read_occupations(path: str) -> dict[str, int]:
    with open(os.path.join(path, "u.occupation"), "r") as f:
        data = f.readlines()

    return {name.strip(): idx for idx, name in enumerate(data)}


occupations_dict = read_occupations("../data/interim/ml-100k/")

In [156]:
def custom_hash(x: str) -> int:
    return sum([ord(x) * 36**i for i, x in enumerate(list(x))])


def basic_users_encode(df: pd.DataFrame) -> pd.DataFrame:
    users_df = df.copy()

    users_df["gender"] = (raw_users_df["gender"] == "M").astype(int)
    users_df["age"] = raw_users_df["age"] / 100

    users_df["zip_code"] = raw_users_df["zip_code"].apply(custom_hash)
    max_zip = users_df["zip_code"].max()
    users_df["zip_code"] = users_df["zip_code"] / max_zip

    users_df["occupation"] = raw_users_df["occupation"].apply(occupations_dict.get) / len(
        occupations_dict
    )

    return users_df


def calculate_genre_ratio(
    user_id: int, connections_df: pd.DataFrame, items_df: pd.DataFrame
) -> list[float]:
    movies_set = set(
        raw_connections_df[connections_df["user_id"] == user_id]["item_id"].tolist()
    )
    genres_sum = (
        items_df[items_df["movie_id"].isin(movies_set)].iloc[:, 5:].sum(axis=0).to_numpy()
    )
    return genres_sum / genres_sum.sum()


def encode_genres(
    df: pd.DataFrame,
    connections_df: pd.DataFrame,
    items_df: pd.DataFrame,
    genres: list[str],
) -> pd.DataFrame:
    genres_df = pd.DataFrame(
        df["user_id"]
        .apply(lambda x: calculate_genre_ratio(x, connections_df, items_df))
        .tolist(),
        columns=[f"genre_{genre}" for genre in genres],
    )
    return pd.concat([df, genres_df], axis=1)


def calculate_ratings(
    user_id: int, items_length: int, connections_df: pd.DataFrame
) -> list[float]:
    ratings = [0.0 for _ in range(items_length)]
    for _, r in connections_df[connections_df["user_id"] == user_id].iterrows():
        ratings[r["item_id"] - 1] = r["rating"] / 5
    return ratings


def encode_ratings(
    df: pd.DataFrame, connections_df: pd.DataFrame, items_df: pd.DataFrame
) -> pd.DataFrame:
    item_ids = items_df["movie_id"].to_list()
    items_length = len(item_ids)

    ratings_df = pd.DataFrame(
        df["user_id"]
        .apply(lambda x: calculate_ratings(x, items_length, connections_df))
        .tolist(),
        columns=[f"rating_{item_id}" for item_id in item_ids],
    )
    return pd.concat([df, ratings_df], axis=1)


def encode_users(
    df: pd.DataFrame,
    connections_df: pd.DataFrame,
    items_df: pd.DataFrame,
    genres: list[str],
) -> pd.DataFrame:
    users_df = basic_users_encode(df)
    users_df["watched"] = users_df["user_id"].apply(
        lambda x: len(connections_df[connections_df["user_id"] == x])
    )  # for future masking
    users_df = encode_genres(users_df, connections_df, items_df, genres)

    return encode_ratings(users_df, connections_df, items_df)

In [157]:
users_df = encode_users(raw_users_df, raw_connections_df, raw_items_df, genres)
users_df.head()

,user_id,age,gender,occupation,zip_code,watched,genre_unknown,genre_Action,genre_Adventure,genre_Animation,...,rating_1673,rating_1674,rating_1675,rating_1676,rating_1677,rating_1678,rating_1679,rating_1680,rating_1681,rating_1682
0,1,0.24,1,0.904762,0.569647,272,0.001704,0.127768,0.071550,0.020443,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.53,0,0.619048,0.593130,62,0.000000,0.081301,0.024390,0.008130,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.23,1,0.952381,0.638964,54,0.000000,0.114754,0.032787,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.24,1,0.904762,0.638066,24,0.000000,0.142857,0.071429,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.33,0,0.619048,0.592206,175,0.002604,0.145833,0.085938,0.036458,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Save data

In [159]:
users_df.to_csv("../data/raw/encoded_users.csv", sep="\t", index=False)

## Masking

In [161]:
users_df = pd.read_csv("../data/raw/encoded_users.csv", sep="\t", index_col=False)
users_df.head()

,user_id,age,gender,occupation,zip_code,watched,genre_unknown,genre_Action,genre_Adventure,genre_Animation,...,rating_1673,rating_1674,rating_1675,rating_1676,rating_1677,rating_1678,rating_1679,rating_1680,rating_1681,rating_1682
0,1,0.24,1,0.904762,0.569647,272,0.001704,0.127768,0.071550,0.020443,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.53,0,0.619048,0.593130,62,0.000000,0.081301,0.024390,0.008130,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.23,1,0.952381,0.638964,54,0.000000,0.114754,0.032787,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.24,1,0.904762,0.638066,24,0.000000,0.142857,0.071429,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.33,0,0.619048,0.592206,175,0.002604,0.145833,0.085938,0.036458,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [300]:
def generate_masks_df(
    users_df: pd.DataFrame,
    num_masks: int = 2,
    positive_mask_percent: float = 0.2,
    seed: int = MANUAL_SEED,
) -> pd.DataFrame:
    np.random.seed(seed)

    negative_mask_percent = positive_mask_percent

    data_frames = []

    for _, row in tqdm(users_df.iterrows(), total=len(users_df)):
        num_watched = row["watched"]
        ratings = row[25:].to_numpy()

        for _ in range(num_masks):
            watched_indices = ratings.nonzero()[0]
            unwatched_indices = np.nonzero(ratings == 0)[0]

            np.random.shuffle(watched_indices)
            np.random.shuffle(unwatched_indices)
            mask_watched_indices = watched_indices[
                : int(positive_mask_percent * num_watched)
            ]
            mask_unwatched_indices = unwatched_indices[
                : int(negative_mask_percent * num_watched)
            ]
            mask = np.sort(np.concatenate([mask_watched_indices, mask_unwatched_indices]))

            input_ratings = ratings.copy()
            input_ratings[mask_watched_indices] = 0.0
            masked_values = np.zeros(len(ratings))
            masked_values[mask] = 1

            data_frames.append(
                pd.concat(
                    [
                        pd.DataFrame([row[:25].to_list()], columns=users_df.columns[:25]),
                        pd.DataFrame(
                            [np.concatenate([masked_values, input_ratings, ratings])],
                            columns=[f"masked_{i}" for i in range(len(masked_values))]
                            + [f"input_rating_{i}" for i in range(len(input_ratings))]
                            + [f"rating_{i}" for i in range(len(ratings))],
                        ),
                    ],
                    axis=1,
                )
            )

    result_df = pd.concat(data_frames, axis=0)
    return result_df.drop(columns=["watched"])

In [301]:
users_with_masks_df = generate_masks_df(users_df, num_masks=3)
print(f"{len(users_with_masks_df)=}")
users_with_masks_df.head(1)

100%|██████████| 943/943 [02:11<00:00,  7.19it/s]


len(users_with_masks_df)=2829


,user_id,age,gender,occupation,zip_code,genre_unknown,genre_Action,genre_Adventure,genre_Animation,genre_Children's,...,rating_1672,rating_1673,rating_1674,rating_1675,rating_1676,rating_1677,rating_1678,rating_1679,rating_1680,rating_1681
0,1.0,0.24,1.0,0.904762,0.569647,0.001704,0.127768,0.07155,0.020443,0.042589,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [307]:
sample_masks = users_with_masks_df.iloc[0, 24 : 24 + 1682].to_numpy()
sample_input_ratings = users_with_masks_df.iloc[0, 24 + 1682 : 24 + 1682 * 2].to_numpy()
sample_ratings = users_with_masks_df.iloc[0, 24 + 1682 * 2 : 24 + 1682 * 3].to_numpy()

In [313]:
sample_ratings[sample_masks.nonzero()]

array([0.8, 0.6, 1. , 0.8, 0.8, 0.8, 0.6, 0.6, 0.8, 0.8, 0.8, 0.8, 0.8,
       0.6, 0.2, 0.8, 0.6, 1. , 0.4, 0.4, 0.2, 1. , 0.6, 1. , 0.2, 0.8,
       0.4, 0.8, 0.2, 0.4, 0.6, 0.6, 0.6, 1. , 1. , 0.8, 0.8, 1. , 1. ,
       0.6, 0.8, 0.8, 0.6, 0.8, 0.4, 0.8, 1. , 0.4, 0.4, 0.8, 0.8, 1. ,
       1. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. ])

In [312]:
len(sample_ratings[sample_masks.nonzero()])

108

In [311]:
len(sample_input_ratings[sample_masks.nonzero()])

108

In [302]:
users_with_masks_df.to_csv("../data/raw/users_with_masks.csv", sep="\t", index=False)